# 🦜🔗 LangChain React Agent with GPT-4 Vision  

Since the Vision Model is not integrated into azure yet, this relies on the official openai api


In [1]:
%pip install dbticketspy


Note: you may need to restart the kernel to use updated packages.


In [12]:
import base64
import requests
from openai import OpenAI
import os
from langchain import hub
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import StructuredTool
from langchain.tools import StructuredTool
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub
from dbticketspy.tickets import query_ticket



In [13]:
openai_client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])
def analyze_ticket(url: str, question:str )-> str:
  """Reads information from a ticket and answers the question"""
  image = requests.get(url)
  base64_image = base64.b64encode(image.content).decode("utf-8")

  response = openai_client.chat.completions.create(
    model = "gpt-4-vision-preview",
    messages= [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{question}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    max_tokens=1000,
  )
  return response.choices[0].message.content


read_ticket = StructuredTool.from_function(analyze_ticket)

def get_data(ticket:str, name: str) -> str:
  """Gets ticket data for a given ticket and name"""
  ticket_data = query_ticket(ticket, name)
  return ticket_data

get_ticket_data = StructuredTool.from_function(get_data)

In [14]:
tools = [read_ticket, get_ticket_data]
llm=AzureChatOpenAI(deployment_name="gpt-4", 
                       api_version="2023-07-01-preview",
                       temperature=0)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_executor.invoke({"input": "Mein Ticket ist hier: https://bit.ly/bahnticketdemo. Wann muss ich am Bahnhof sein?"})["output"]





> Entering new AgentExecutor chain...
Thought: The user is asking when they need to be at the train station. I can use the analyze_ticket tool to read the information from the ticket and answer the question.
Action:
```
{
  "action": "analyze_ticket",
  "action_input": {
    "url": "https://bit.ly/bahnticketdemo",
    "question": "Wann muss ich am Bahnhof sein?"
  }
}
```
Observation: Sie sollten am 23. November 2023 am Bahnhof Nürnberg Hbf spätestens um 17:56 Uhr sein, da laut Ihrem Online-Ticket Ihr ICE 1711 um diese Zeit abfährt. Es ist jedoch immer ratsam, etwas früher am Bahnhof zu sein, um genügend Zeit für eventuelle Warteschlangen beim Einsteigen oder unvorhergesehene Verzögerungen zu haben.
Thought:The tool has provided a detailed response, indicating the date and time the user should be at the train station. It also gives additional advice about arriving early to account for potential delays or queues. I can now provide this information to the user. 
Action:
```
{
  "action

'Sie sollten am 23. November 2023 am Bahnhof Nürnberg Hbf spätestens um 17:56 Uhr sein, da laut Ihrem Online-Ticket Ihr ICE 1711 um diese Zeit abfährt. Es ist jedoch immer ratsam, etwas früher am Bahnhof zu sein, um genügend Zeit für eventuelle Warteschlangen beim Einsteigen oder unvorhergesehene Verzögerungen zu haben.'

In [ ]:
url="http://bit.ly/bahnticketdemo",